# Ejercicio N°1: Análisis de fletes aéreos de vacunas COVID-19

## Instrucciones:
1. Leer en Colab el archivo "flete-aereo-vacunas-covid19-al-2021-06-28.xlsx"
2. Realizar tareas de limpieza y/o adecuación del dataset necesarias

## Preguntas a resolver:
1. ¿Cuál fue el número de vuelo que realizó una mayor cantidad de fletes?
   (Sugerencia: utilizar el método value_counts() de Pandas)

2. ¿Cuántos registros no contienen información sobre el número de vuelo?

3. Calcular el promedio de lo facturado en todos los vuelos realizados,
   considerando únicamente los registros de viajes facturados en USD
   según la columna factura_moneda_monto.

4. ¿Qué porcentaje de los vuelos realizados tuvieron como origen a Rusia o a China?

5. ¿Cuál es el vuelo más reciente de los que se tiene registro en el dataset?
   ¿Cuántos días transcurrieron entre el primer y el último vuelo realizados?

6. Escribir el archivo en formato .parquet.

In [ ]:
import pandas as pd
data = pd.read_excel('/content/flete-aereo-vacunas-covid19-al-2021-06-28.xlsx', header=4, nrows=100)
data = data.rename(columns={'vuelo ': 'vuelo'})
pd.set_option('display.max_columns', None)

In [ ]:
print(list(data.columns))
# Explicitar tipo de datos de un csv con un parámetro en el comando de lectura
# pd.read_csv('listings.csv', dtype = {'price': 'float'})
print(data.info())
print(data.info())  # Ver tipos de datos y valores nulos
print(data.head())  # Ver las primeras filas
print(data.describe())  # Estadísticas generales
nan_count = data.isnull().sum().sum()
print(nan_count)
notnull_count = data.notnull().sum()
print(notnull_count)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   organismo                       66 non-null     object        
 1   expediente                      66 non-null     object        
 2   financiamiento                  66 non-null     object        
 3   acto_conclusión                 66 non-null     object        
 4   descripción                     66 non-null     object        
 5   prestador                       66 non-null     object        
 6   CUIT                            66 non-null     object        
 7   estado                          66 non-null     object        
 8   factura_nro                     66 non-null     object        
 9   factura_moneda_monto            64 non-null     object        
 10  guía_nro                        66 non-null     object        
 11  guía_mon

In [ ]:
print(list(data.columns))

['organismo', 'expediente', 'financiamiento', 'acto_conclusión', 'descripción', 'prestador', 'CUIT', 'estado', 'factura_nro', 'factura_moneda_monto', 'guía_nro', 'guía_moneda_monto', 'fecha_guía', 'vuelo', 'bienes_transportados', 'proveedor_bienes_transportados', 'operación', 'comprador_donante', 'origen']


1. ¿Cuál fue el número de vuelo que realizó una mayor cantidad de fletes? (Sugerencia: utilizar el método value_counts() de Pandas)

In [ ]:
conteo_vuelos = data["vuelo"].value_counts()
print('El vuelo más frecuente es el:', conteo_vuelos.idxmax())

El vuelo más frecuente es el: 1061


2. ¿Cuántos registros no contienen información sobre el número de vuelo?

In [ ]:
cantidad_vuelos_sin_informacion = data["vuelo"].isna().sum()
print(cantidad_vuelos_sin_informacion, 'registros no contienen información sobre el número de vuelo.')

3 registros no contienen información sobre el número de vuelo.


3. Calcular el promedio de lo facturado en todos los vuelos realizados, considerando únicamente los registros de viajes facturados en USD según la columna factura_moneda_monto.

In [ ]:
def limpiar_montos(registro):
  """
  (1) toma un registro, pasado con .apply(limpiar_montos)
  (2) si es Nan, devuelve NaN
  (3) si no es nan, divide por espacios por ejemplo "USD" y "234.234,44"
  (4) quita los puntos.
  (5) reemplaza las comas por puntos
  (6) convierte el valor que debería haber quedado por ejemplo "234234.44"
  (7) finalmente retorna el registro limpio y en formato float para hacer cálculos
  es importante que antes le pasemos los registros de USD, sino se perdería
  la información del tipo de moneda.
  """
  #if pd.isna(registro):
  #  return float('nan')
  partes = registro.split()
  monto = partes[1]
  monto = monto.replace(".", "")
  monto = monto.replace(",", ".")
  return float(monto)

In [ ]:
facturas_USD = data[data['factura_moneda_monto'].str.contains('USD', na=False)]
facturas_USD_limpios = facturas_USD['factura_moneda_monto'].apply(limpiar_montos)
"""
me di cuenta acá que está bueno que conserve el index original porque de esta manera podemos comparar el resultado de la transformación con
los valores del dataframe original

para comparar la columna original con la final
data_comparacion = pd.DataFrame({
    "factura_moneda_monto": data['factura_moneda_monto'],
    "factura_monto_limpio": facturados_en_USD_limpios
})
pd.set_option('display.max_rows', None)
print(data_comparacion)
"""
print('El promedio de lo facturado en USD es: ', facturas_USD_limpios.mean())

El promedio de lo facturado en USD es:  208566.49125


4. ¿Qué porcentaje de los vuelos realizados tuvieron como origen a Rusia o a China?

In [ ]:
valores_unicos = data["origen"].unique()
print("Valores únicos en la columna origen:", valores_unicos)

origen_china_rusia = data[data["origen"].str.contains("China|Rusia", case=False, na=False)]["origen"]
cant_vuelos_totales = len(data)
porcentaje_china_rusia = len(origen_china_rusia)/cant_vuelos_totales * 100
print(f'El porcentaje de vuelos con ORIGEN de China o Rusia es de {porcentaje_china_rusia:.2f}%')

Valores únicos en la columna origen: ['Rusia' 'India' 'China' 'Paises Bajos' 'china']
El porcentaje de vuelos con ORIGEN de China o Rusia es de 96.97%


¿Cuál es el vuelo más reciente de los que se tiene registro en el dataset? ¿Cuántos días transcurrieron entre el primer y el último vuelo realizados?

In [ ]:
fecha_max = data['fecha_guía'].max().date()
print(f'El vuelo más reciente en el dataset es de {fecha_max}.')

vuelo_mas_reciente = data['fecha_guía'].max()
vuelo_mas_antiguo = data['fecha_guía'].min()
diferencia_de_tiempo = vuelo_mas_reciente - vuelo_mas_antiguo
print(f"Entre el primer y el último vuelo transcurrieron {diferencia_de_tiempo.days} días.")

El vuelo más reciente en el dataset es de 2021-06-22.
Entre el primer y el último vuelo transcurrieron 181 días.


6. Escribir el archivo en formato .parquet.

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

tabla = pa.Table.from_pandas(data)
pq.write_table(tabla, 'vuelos_covid.parquet')

ArrowInvalid: ("Could not convert 'QR8351/16-QR8155/16' with type str: tried to convert to int64", 'Conversion failed for column vuelo with type object')

In [ ]:
# Hay conflicto con el tipo de dato de la columan "vuelo". Todos son números menos este que indica arriba que es 'QR8351/16-QR8155/16'
# Por esto hacemos uso de astype(str) para convertir todo a string.
data["vuelo"] = data["vuelo"].astype(str)
tabla = pa.Table.from_pandas(data)
pq.write_table(tabla, 'vuelos_covid.parquet')